In [10]:
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from commonfunctions import *
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.color import rgb2gray
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.transform import resize
import cv2
import os
from sklearn.svm import  SVC
import pickle
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
# Get Features From Image Using HOG
def getFeature (image):
    # Detect Face Using OpenCV
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(image, 1.1, 4)
    if len(faces) != 0:
        for (x, y, w, h) in faces:
            image = image[y:y+h,x:x+w]     
            # Convert Image TO Gray Image If its RGB Image
            if(len(image.shape)==3):
                image = rgb2gray(image)
            # Convert Float64 to Uint8
            if(image.dtype == 'float64'):
                image = (image*255).astype('uint8')
            if(image.shape[0] != 0 and image.shape[1] != 0):
                # Resize Image to make all Images with the same dimensions
                image = resize(image, (128, 128))
                if(image.dtype == 'float64'):
                    image = (image*255).astype('uint8')
                if(image.shape[0] != 0 and image.shape[1] !=0):
                    # Get HOG Histogram
                    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                                        cells_per_block=(2, 2), visualize=True)
                    return fd
    

In [17]:
# Load Trained Model
filename = 'model1.sav'
SVM_Modle = pickle.load(open(filename, 'rb'))

In [18]:
# Use Camera

# Emotions Labels
emotions = ['anger','disgust','fear','happiness','neutral','sadness','surprise']
vid = cv2.VideoCapture(0)
text = 'not recognized'
while(True):
    ret, frame = vid.read(0)
    # Text Shown in the corner (Predicted Emotion)
    cv2.putText(frame, text, (50, 50-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    X = getFeature(image=frame)
    if(X is not None):
        X = X.reshape(1,-1)
        cv2.imshow('frame', frame)
        # Get Prediction
        predictions = SVM_Modle.predict(X.reshape(1,-1))
        text = emotions[int(predictions[0])-1]
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()